In [49]:
import java.nio.file.Files
import java.nio.file.Path

data class BenchmarkData(val name: String, val score: Double, val error: Double) {
    val type = getType(name)
    override fun toString(): String = "$name: $score ± $error ns/op"
}

enum class Type { Int, Long, Float, Double }

fun getType(name: String) = when {
    name.contains("Float") -> Type.Float
    name.contains("Int") -> Type.Int
    name.contains("Double") -> Type.Double
    name.contains("Long") -> Type.Long // Long must be the last as it is contained in other test names
    else -> error("Unknown type: ${name}")
}

fun readHotspotBenchmarkData(benchmarkName: String) = Files.readAllLines(Path.of("build", "results", "jmh", "results.txt"))
    .mapNotNull {
        val r = Regex("$benchmarkName\\.([^ ]+) +avgt +\\d+ +([0-9.]+) +± +([0-9.]+) +ns/op")
        val match = r.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            name = match.groups[1]!!.value,
            score = match.groups[2]!!.value.toDouble(),
            error = match.groups[3]!!.value.toDouble(),
        )
    }.sortedBy { it.type }
(readHotspotBenchmarkData("Wolf3dBenchmark") + readHotspotBenchmarkData("FibonacciBenchmark")).joinToString("\n")

baselineFloat: 3.3197973525E7 ± 154895.663 ns/op
inlineFloat: 3.2854524192E7 ± 187687.244 ns/op
longPackFloat: 5.6029136127E7 ± 49003.45 ns/op
mutableRefFloatIn1Long: 6.4553914124E7 ± 389302.707 ns/op
mutableRefFloatIn2Longs: 3.9385519158E7 ± 228039.197 ns/op
valueFloat: 2.5007839893E7 ± 61239.655 ns/op
valueInlineFloat: 2.5033691073E7 ± 43272.321 ns/op
valuePreserveBoxFloat: 2.5045356928E7 ± 39915.856 ns/op
baselineDouble: 3.4183342161E7 ± 179414.77 ns/op
inlineDouble: 3.4746828463E7 ± 215704.987 ns/op
mutableRefDouble: 4.0317501886E7 ± 565618.853 ns/op
valueDouble: 2.4596098315E7 ± 40779.822 ns/op
valueInlineDouble: 2.4578034483E7 ± 57816.984 ns/op
valuePreserveBoxDouble: 2.4637674513E7 ± 46559.077 ns/op

In [50]:
%use lets-plot

In [51]:
import org.jetbrains.letsPlot.intern.PosKind
import org.jetbrains.letsPlot.intern.StatKind
import org.jetbrains.letsPlot.intern.layer.PosOptions
import org.jetbrains.letsPlot.intern.layer.StatOptions

fun makePlot(name: String, platform: String, benchmarkLines: List<BenchmarkData>) {
    if (benchmarkLines.isEmpty()) return
    val names = benchmarkLines.map { it.name }
    val scores = benchmarkLines.map { it.score }
    val scoresMinusErrors = benchmarkLines.map { it.score - it.error / 2 }
    val scoresPlusErrors = benchmarkLines.map { it.score + it.error / 2 }
    val baseline = benchmarkLines.filter { it.name.startsWith("baseline") }.groupBy { it.type }.mapValues { (_, v) -> v.single() }
    val maxScore = scoresPlusErrors.max()
    val plot = letsPlot() +
            labs("$name ($platform)", fill = "Benchmark", y = "ns/op") +
            geomBar(stat = StatOptions(kind = StatKind.IDENTITY)) {
                x = names
                y = scores
                fill = names
            } +
            geomErrorBar(
                width = 0.5,
                position = PosOptions(kind = PosKind.DODGE),
                size = 0.6
            ) {
                x = names
                ymin = scoresMinusErrors
                ymax = scoresPlusErrors
            } +
            geomText(
                stat = StatOptions(kind = StatKind.IDENTITY),
                position = positionStack(vjust = 1.0),
                size = 4.5,
            ) {
                x = names
                y = scores.map {
                    val threshold = 0.08
                    if (it < maxScore * threshold) it + threshold * maxScore / 2 else it / 2
                }
                label = benchmarkLines.map {
                    val score = it.score / baseline[it.type]!!.score
                    val error = it.error / baseline[it.type]!!.score
                    "${String.format("%.0f", score * 100)}%\n±\n${String.format("%.1f", error * 100)}%\n${it.type}"
                }
            } +
            scaleXDiscrete(
                labels = names,
                name = "",
                limits = names + listOf(""),
                breaks = names.toList()
            ) +
            scaleYContinuous(limits = 0 to maxScore, name = "ns/op") +
            theme(plotTitle = elementText(hjust = 0.5)).legendPositionRight() +
            ggsize(1000, 800)
    ggsave(plot = plot, filename = "$name ($platform).svg")
    plot.show()
}


In [52]:
makePlot("Wolf3d", "Hotspot", readHotspotBenchmarkData("Wolf3dBenchmark"))
makePlot("Ackermann", "Hotspot", readHotspotBenchmarkData("AckermannBenchmark"))
makePlot("BoxRecreation", "Hotspot", readHotspotBenchmarkData("BoxRecreationBenchmark"))

In [53]:
import java.io.FileFilter
import kotlin.io.path.listDirectoryEntries

fun readArtBenchmarkData(benchmarkName: String) = Path.of("android-benchmark/build/outputs/androidTest-results/connected")
    .toFile().listFiles(FileFilter { it.isDirectory })!!.single()
    .listFiles { dir, name -> name.startsWith("logcat-org.jetbrains.") }!!.flatMap { it.readLines() }
    .mapNotNull {
        val pattern = Regex("^\\d{2}-\\d{2} +\\d+:\\d+:\\d+.\\d+ +\\d+ +\\d+ +I +Benchmark: +$benchmarkName\\.([^\\[]+)\\[Metric \\(timeNs\\) +results: +median +([^,]+), +min [^,]+, +max [^,]+, +standardDeviation: +([^,]+), +.*")
        val match = pattern.matchEntire(it) ?: return@mapNotNull null
        BenchmarkData(
            name = match.groups[1]!!.value,
            score = match.groups[2]!!.value.toDouble(),
            error = match.groups[3]!!.value.toDouble(),
        )
     }.let { lines ->
         val indexes = readHotspotBenchmarkData(benchmarkName).mapIndexed { index, benchmarkData -> benchmarkData.name to index }.toMap()
         if (lines.all { it.name in indexes }) lines.sortedBy { indexes[it.name]!! } else lines.sortedBy { it.type }
     }
(readArtBenchmarkData("Wolf3dBenchmark") + readArtBenchmarkData("FibonacciBenchmark")).joinToString("\n")

baselineFloat: 3.95458115E8 ± 7258662.437255299 ns/op
inlineFloat: 6.781380575E8 ± 2.5450937315296076E7 ns/op
longPackFloat: 1.63413904E8 ± 147385.38704067026 ns/op
mutableRefFloatIn1Long: 2.21802289E8 ± 1332887.060087233 ns/op
mutableRefFloatIn2Longs: 1.409159425E8 ± 600245.721043732 ns/op
valueFloat: 3.607298075E8 ± 5847946.481234766 ns/op
valueInlineFloat: 2.4474173125E7 ± 31436.598048008324 ns/op
valuePreserveBoxFloat: 3.548367305E8 ± 6702955.516523047 ns/op
baselineDouble: 4.81480923E8 ± 6803477.62664473 ns/op
inlineDouble: 1.0713920375E9 ± 1.261424518724434E7 ns/op
mutableRefDouble: 1.478356155E8 ± 254485.43250355776 ns/op
valueDouble: 4.38175519E8 ± 7438010.360965897 ns/op
valueInlineDouble: 2.5891435833333336E7 ± 43994.457027732926 ns/op
valuePreserveBoxDouble: 5.368072305E8 ± 1.1805937592653433E7 ns/op

In [54]:
makePlot("Wolf3d", "ART", readArtBenchmarkData("Wolf3dBenchmark"))
makePlot("Ackermann", "ART", readArtBenchmarkData("AckermannBenchmark"))
makePlot("BoxRecreation", "ART", readArtBenchmarkData("BoxRecreationBenchmark"))